# Tip Calculator

In [239]:
import pandas as pd
from datetime import datetime

In [240]:
class WorkDay:
    def __init__(self, day, hours):
        self.day = datetime.strptime(day[3:], "%d.%m.%Y")
        self.hours = self._parse_hours(hours)
    
    def add(self, workday):
        if (self.day != workday.day):
            raise ValueError('Days of workdays must match to sum up work hours!')
        else:
            self.hours += workday.hours

    def _parse_hours(self, hours):
        hours, minutes = map(int, hours.split(':'))
        return round(hours + (minutes / 60), 2)      
    
    def __repr__(self):
        return f"{self.day}: {self.hours}"
    
class Employee:
    def __init__(self, name, work_days):
        self.name = name
        self.work_days = work_days

    def __repr__(self):
        work_days = '\n'.join('\t' + str(work_day) for work_day in self.work_days)
        return f"{self.name}: \n{work_days}"

def parse_hours_for(name):
    sheet = xl.parse(name, header=None)
    header = sheet[sheet[0] == 'Tag'].index[0]

    data = xl.parse(name, header=header)
    data = data[['Tag', 'Startzeit', 'Endzeit', 'Pause (min)', 'Dauer netto (h)']]
    data = data[data['Startzeit'].notnull()]
    
    work_days = {}
    for _, row in data.iterrows():
        work_day = WorkDay(day=row['Tag'], hours=row['Dauer netto (h)'])
        if work_day.day in work_days:
            work_days[work_day.day].add(work_day)
        else:
            work_days[work_day.day] = work_day

    employee = Employee(name=name, work_days=work_days.values())

    return employee


In [241]:
class DailyWorkHours:
    def __init__(self, tip):
        self.tip = tip
        self.work_hours = {}

    def total_hours(self):
        return round(sum(self.work_hours.values()), 2)
    
    def total_tip_per_hour(self):
        return round(self.tip / self.total_hours(), 2);
    
    def hours_from_decimal(self, decimal):
        hours = int(decimal)
        minutes = round((decimal - hours) * 60)
        return f"{hours:02}:{minutes:02}"

    def __repr__(self):
        total_hours = self.total_hours()
        tip_per_hour = self.total_tip_per_hour()
        work_hours = '\n'.join([f'\t{name}:\n\t\thours {hours}\n\t\thours (decimal) {self.hours_from_decimal(hours)}\n\t\ttip {round(hours * tip_per_hour, 2)}' for name, hours in self.work_hours.items()])
        return f'total tip {self.tip}, total hours {total_hours}, tip per hour {tip_per_hour}\n{work_hours}'
    
class WeeklyTips:
    def __init__(self, daily_tips, employees_work_hours):
        days = sorted({work_day.day for employee in employees_work_hours for work_day in employee.work_days})
        self.weekly_work_times = {day: DailyWorkHours(tip) for day, tip in zip(days, daily_tips)}
        for employee in employees_work_hours:
            for work_day in employee.work_days:
                daily_work_hours = self.weekly_work_times[work_day.day]
                daily_work_hours.work_hours[employee.name] = work_day.hours

    def __repr__(self):
        return '\n'.join([f'{day.strftime("%d.%m.%Y")}: {daily_work_times}' for day, daily_work_times in self.weekly_work_times.items()])

In [242]:
# Change the variable 'file' and 'daily_tips'
file = 'sample01' # input file name
path = (f'./data/{file}.xlsx')
daily_tips = [112.00, 177.71, 155.20, 96.22, 143.50, 181.50, 160.90] # tips MO-SO

xl = pd.ExcelFile(path)
employees = xl.sheet_names
employees.remove('Übersicht')
employees_work_hours = [parse_hours_for(name) for name in employees]

weekly_work_times = WeeklyTips(daily_tips=daily_tips, employees_work_hours=employees_work_hours)

In [243]:
all_employees = [
    "Alicia Anderson", "Emilie Appl", "Mina Aryal", "Nadine Aschenbrenner",
    "Omar Massire Balde", "Acelya Basili", "Fabrizio Bellia", "Filippo Bieringer",
    "Anna Sophie Blessing", "Sabullah Bullach", "Manuel Bär", "Nepomuk Böhm",
    "Tom Colombo", "Annika Ebel", "Stina Ebel", "Aaron Eißner", "Jasmin Engelberger",
    "Alicia Ekoos", "Paul Firmhofer", "Markus Fuchs", "Marie Gaschler", "Lisa Gierer",
    "Falk Golla", "Dodo Goßner", "Ekaterina Grashchenko", "Laura Greidenweiß",
    "Karen Gronbach", "Miriam Guggenberger", "Svenja Halamek", "Nicklas Herold",
    "Katrin Hiller", "Marie Jaussi", "Kuda Kabylbekov", "Angela Knoll",
    "Claudine-Sophie Kopetz", "Elina Kuldeva", "Philippe Leonpacher",
    "Moritz Liederscheidt", "Marlene Lind", "Anton Machowski", "Marleen Mulzer",
    "Simon Männlein", "Sarah-Michelle Müller", "Jannik Mülhaupt", 
    "Okechulowu (Anselem) Ohaebism", "Philine Ostermayer", "Damiano Parziale",
    "Alyah Pattis", "Luca Principi", "Reshimi Raj Aryal", "Lilian Reiner",
    "Emilia Sachsen-Coburg", "Luca Sawallisch", "Pia Schmolly", "Kilian Schugsties",
    "Seyed Mohammad", "Mauro Sirigu", "Matthias Strobel", "Tanja Stürhof",
    "Luca Tegeder", "Fenny Tran", "Bent van Zon", "Charlotte von Schröder",
    "Jana Wegenke", "Lilli Wittig", "Jennifer Wöhrlin", "Fabian Zink", "Anna Graf"
]
all_employees.sort()

In [244]:
from openpyxl import Workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side
import string

wb = Workbook()
ws = wb.active

cols = list(string.ascii_uppercase) + [f'{letter}{next_letter}' for letter in string.ascii_uppercase for next_letter in string.ascii_uppercase[:1]]
column_width = 20
employee_start_row = 6

def set_column_width(col_width, cols):
    for col in cols:
        ws.column_dimensions[col].width = col_width

def set_borders(until_row, cols):
    border_style_right = Border(
        right=Side(border_style='thin', color='000000')
    )

    for row in range(1, until_row):
        for col in cols:
            ws[f'{col}{row}'].border = border_style_right

    border_style_all = Border(
        top=Side(border_style='thin', color='000000'),
        right=Side(border_style='thin', color='000000'),
        bottom=Side(border_style='thin', color='000000'),
        left=Side(border_style='thin', color='000000'),
    )

    for col in cols:
        ws[f'{col}5'].border = border_style_all

    border_style_top = Border(
        top=Side(border_style='thin', color='000000')
    )

    for col in cols:
        ws[f'{col}{until_row}'].border = border_style_top

def write_employee_col(employees):
    ws['A5'] = 'Mitarbeiter'
    ws['A5'].font = Font(bold=True)

    for row, employee in enumerate(employees, start=employee_start_row):
        ws[f'A{row}'] = employee

def write_day_header(col, day, total_tip, total_hours, total_tip_per_hour):
    col_index = cols.index(col)
    rel_A = cols[col_index]
    rel_B = cols[col_index+1]
    rel_C = cols[col_index+2]

    ws.merge_cells(f'{rel_A}1:{rel_C}1')
    ws[f'{rel_A}1'] = day
    ws[f'{rel_A}1'].font = Font(size=13, bold=True)
    ws[f'{rel_A}1'].alignment = Alignment(horizontal='center', vertical='center')

    ws.merge_cells(f'{rel_A}2:{rel_B}2')
    ws[f'{rel_A}2'] = 'Trinkgeld (gesamt)'
    ws[f'{rel_C}2'] = total_tip
    ws[f'{rel_C}2'].number_format = '"€"#,##0.00'

    ws.merge_cells(f'{rel_A}3:{rel_B}3')
    ws[f'{rel_A}3'] = 'Stunden (gesamt)'
    ws[f'{rel_C}3'] = total_hours
    ws[f'{rel_C}3'].alignment = Alignment(horizontal='right')

    ws.merge_cells(f'{rel_A}4:{rel_B}4')
    ws[f'{rel_A}4'] = 'Trinkgeld pro Stunde'
    ws[f'{rel_C}4'] = total_tip_per_hour

    ws[f'{rel_A}5'] = 'Stunden'
    ws[f'{rel_A}5'].font = Font(bold=True)

    ws[f'{rel_B}5'] = 'Stunden (Dezimal)'
    ws[f'{rel_B}5'].font = Font(bold=True)

    ws[f'{rel_C}5'] = 'Trinkgeld'
    ws[f'{rel_C}5'].font = Font(bold=True)

def write_day_row(col, row, hours, hours_decimal, tip):
    col_index = cols.index(col)
    rel_A = cols[col_index]
    rel_B = cols[col_index+1]
    rel_C = cols[col_index+2]

    ws[f'{rel_A}{row}'] = hours_decimal
    ws[f'{rel_A}{row}'].alignment = Alignment(horizontal='right')
    ws[f'{rel_B}{row}'] = hours
    ws[f'{rel_C}{row}'] = tip
    ws[f'{rel_C}{row}'].number_format = '"€"#,##0.00'

def write_day_col(col, day_title, day):
    write_day_header(col, day_title, day.tip, day.total_hours(), day.total_tip_per_hour())

    for row, name in enumerate(all_employees, start=6):
        if name in day.work_hours:
            hours = day.work_hours[name]
            hours_decimal = day.hours_from_decimal(hours)
            tip = round(hours * day.total_tip_per_hour(), 2)
            write_day_row(col, row, hours, hours_decimal, tip)

def write_days(days):
    for col, day_title, day in days:
        write_day_col(col, day_title, day)

set_column_width(column_width, cols[:cols.index('W')])
set_borders(len(all_employees) + 6, cols[:cols.index('W')])

weekly_work_times_sorted = [daily_work_hours for _, daily_work_hours in sorted(weekly_work_times.weekly_work_times.items())]

write_employee_col(all_employees)

days = zip(
    ['B', 'E', 'H', 'K', 'N', 'Q', 'T'],
    ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag'],
    weekly_work_times_sorted
)

write_days(days)

wb.save(f'data/{file}_output.xlsx')
